In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def iterative_classifier(sampleName):
    """
    This function starts with the PixelClassifier model's classifications, then randomly walks from pixel-to-pixel to
    compare each pixel to its neighbors, using that to update the classification. It then compares the new set of
    classifications to the old classifications. If they differ by more than a negligible amount, the process starts
    again until the differences are within a tolerance.
    
    inputs: sampleName - a string with the name of the sample, excluding the filetype extension, which is assumed to
                    be .txt
            ;sdjfos;dnf
            
    outputs:
    """
    
    scan = np.loadtxt('../Data/AFM/AggregatedData/%s.txt'% (sampleName))
    
    x2, z = scan.shape
    x = y = int(math.sqrt(x2))
    scan = scan.reshape((x, y, z))
    
    for i in range(x):         #initialize the classifications to the PixelClassifier identities
        for j in range(y):
            classifications[i, j] = scan[i, j, 5]
            
    